In [1]:
from investment.models import Budget, Transaction, BudgetAmount
from datetime import datetime

Populate the BudgetAmount model

In [2]:
budget_all_items = Budget.objects.all()

In [3]:
budget_amount = BudgetAmount.objects.all()

In [4]:
for amount in budget_amount:
    print(amount.get_amount())

2000.00
7032.60
200.00
1.60
166.15
0
400.00
25.00
300.00
1000.00
320.00
500.00
150.00
80.00
200.00
250.00
168.29
451.00
277.74
300.00
34.49
500.00
240.00
6.89
30.00
2500.00
90.00
200.00
300.00
1000.00
1000.00
120.00
2800.00
400.00
300.00
300.00
1000.00
150.00
120.00
10000.00
7000.00
807.00


In [10]:
for budget_item in budget_all_items:
    # create budget amounts
    # budget_item.set_budget_amount(budget_item.amount, datetime(2022, 1, 1))
    # retreive budget amounts
    print(budget_item)
    print(budget_item.current_amount(evaluation_date="2024-12-12"))

Activité Enfants Variable 2000.0
2000.00
Allocation canadienne pour enfants Fixe 7032.6
7032.60
Allocation famille QC Fixe 200.0
200.00
Apple iCloud Fixe 1.6
1.60
assurances Fixe 166.15
166.15
BS Fixe 1050.0
807.00
Cadeau Variable 400.0
400.00
Cheveux Variable 25.0
25.00
CLub TRI PAT Variable 300.0
300.00
Course Geneviève Variable 1000.0
1000.00
Dentiste Enfants Fixe 320.0
320.00
Dentiste Genevieve Fixe 500.0
500.00
Épicerie Variable 150.0
150.00
Estheticienne Variable 80.0
80.00
frais de garde Fixe 200.0
200.00
Frais scholaires Variable 250.0
250.00
Hydro Fixe 168.29
168.29
Hypothèque Fixe 451.0
451.00
Immatriculations Fixe 277.74
277.74
imprévues Variable 300.0
300.00
Internet Fixe 34.49
34.49
Massage Variable 500.0
500.00
Massages Variable 240.0
240.00
Netflix Fixe 6.89
6.89
Nourriture Cochons Variable 30.0
30.00
Paye Fixe 2500.0
2500.00
Permis de conduire Fixe 90.0
90.00
Pharmacie Variable 200.0
200.00
pneus Fixe 300.0
300.00
Remboursement d'impôt Variable 1000.0
1000.00
Restaurant

Update BS

In [6]:
bs_budget_item = Budget.objects.get(spending_item_name="BS")

In [7]:
bs_budget_item.set_budget_amount(807.00, "2024-03-01")

In [ ]:
def budget_item_transaction_monthly_recon(request, budget_item, date_dict):
    """
    Generate a monthly reconciliation report for a specific budget item.

    Parameters:
    - request (HttpRequest): The HTTP request object.
    - budget_item (Budget): The budget item for which the reconciliation report is generated.
    - date_dict (dict): A dictionary containing date-related information including:
        - 'b' (datetime): Start date of the current month.
        - 'e' (datetime): End date of the current month.
        - 'previous_month_start_date' (datetime): Start date of the previous month.
        - 'previous_month_eom_date' (datetime): End of the previous month.
        - 'next_month_start_date' (datetime): Start date of the next month.
        - 'next_month_eom_date' (datetime): End of the next month.
        - 'evaluation_date' (datetime): Date for which the evaluation is performed.
        - 'boy' (datetime.date): Beginning of the year.

    Returns:
    - context (dict): A dictionary containing the following keys:
        - 'transaction_total' (float): Total amount spent in the current month for the budget item.
        - 'transactions' (QuerySet): Queryset of transactions in the current month for the budget item.
        - 'budget_item' (Budget): The budget item being reconciled.
        - 'previous_month_transfer' (float): Amount carried over from the previous month.
        - 'previous_month_exists' (bool): Indicates whether transactions exist in the previous month.
        - 'next_month_exists' (bool): Indicates whether transactions exist in the next month.
        - 'previous_month_url' (str): URL to view transactions for the previous month.
        - 'next_month_url' (str): URL to view transactions for the next month.
        - 'difference' (float): Difference between the budgeted amount plus carried over amount 
          and the total spent in the current month.

    This function generates a reconciliation report for a specific budget item for the current month,
    including information such as total spent, transactions, carried over amount, and budgeted amount.
    """


    
    # Combine queries for last month and current month
    # from .models import Transaction
    all_transactions_as_of_last_month = Transaction.objects.filter(
        # user__id=request.user.id,
        category__id=budget_item.id,
        date__lte=date_dict['previous_month_eom_date']
    ).order_by('date')

    all_transactions_as_of_this_begining_of_year = Transaction.objects.filter(
        # user__id=request.user.id,
        category__id=budget_item.id,
        date__lt=date_dict['boy']
    ).order_by('date')

    this_month_transaction = Transaction.objects.filter(
        # user__id=request.user.id,
        category__id=budget_item.id,
        date__range=(date_dict['b'], date_dict['e'])
    ).order_by('date')

    # Check for next and previous month existence
    next_month_transactions_exists = date_dict["evaluation_date"] < timezone.make_aware(datetime.today())
    previous_month_transactions_exists = date_dict["evaluation_date"] > timezone.make_aware(datetime(date_dict["evaluation_date"].year, 1, 1))

    # Calculate total_spent and current_month_transactions_total directly in the database
    total_spent_last_year = all_transactions_as_of_this_begining_of_year.aggregate(total_spent=Sum('amount'))['total_spent'] or 0
    total_spent = all_transactions_as_of_last_month.aggregate(total_spent=Sum('amount'))['total_spent'] or 0
    current_month_transactions_total = this_month_transaction.aggregate(current_month_total=Sum('amount'))['current_month_total'] or 0

    # Calculate total_budgeted once and reuse
    total_budgeted = budget_item.calculate_amount() * date_dict["previous_month_eom_date"].month if date_dict["previous_month_eom_date"].month != 12 else 0

    # Calculate carried_over_amount without rounding first
    carried_over_amount = total_budgeted - (total_spent -  total_spent_last_year)

    context = {
        'transaction_total': round(current_month_transactions_total, 2),
        'transactions': this_month_transaction,
        'budget_item': budget_item,
        "previous_month_transfer": round(carried_over_amount, 2),
        'previous_month_exists': previous_month_transactions_exists,
        'next_month_exists': next_month_transactions_exists,
        'previous_month_url': reverse('transactions-by-budget-titem-view', kwargs={'budget_item_id': budget_item.id}),
        'next_month_url': reverse('transactions-by-budget-titem-view', kwargs={'budget_item_id': budget_item.id}),
        "difference": round(budget_item.calculate_amount() + carried_over_amount - current_month_transactions_total, 2)
    }

    return context
